In [4]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import h5py

In [18]:
zones = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

borders = [
    ["AT", "CZ"], ["AT", "DE_LU"], ["AT", "HU"], ["AT", "SI"], ["BE", "FR"],
    ["BE", "NL"], ["CZ", "DE_LU"], ["CZ", "PL"], ["CZ", "SK"], ["DE_LU", "FR"],
    ["DE_LU", "NL"], ["HR", "HU"], ["HR", "SI"], ["HU", "SK"], ["HU", "RO"],
    ["BE", "DE_LU"], ["DE_LU", "PL"], ["HU", "SI"], ["PL", "SK"], ["AT", "SK"],
]

In [16]:
df_grid = pd.read_excel('./grid.xlsx', index_col=0)

In [7]:
start_date_feb = pd.Timestamp('20230201')
end_date_feb = pd.Timestamp('20230301')

In [36]:
hf_d = h5py.File("./border_distance_matrix.jld", "r")
hf_d_c = h5py.File("./border_distance_c_matrix.jld", "r")

border_distance = np.transpose(hf_d.get("data"))
border_distance_c = np.transpose(hf_d_c.get("data"))

In [11]:
df_ptdf_obs = pd.read_excel('./ptdf_z_obs.xlsx', index_col=0)

In [12]:
df_ptdf_forecasts = []
print('alpha 10')
df_ptdf_forecasts.append(pd.read_excel('./ptdf_z_calc_nuts.xlsx', index_col=0))
print('alpha 5')
df_ptdf_forecasts.append(pd.read_excel('./ptdf_z_calc_nuts_feb_5.xlsx', index_col=0))
print('naive')
df_ptdf_forecasts.append(pd.read_excel('./ptdf_z_naive.xlsx', index_col=0))

alpha 10
alpha 5
naive


In [14]:
df_np_observed = pd.read_excel("../cost_curves/data/net_positions.xlsx", index_col=0)
df_np_observed = df_np_observed.loc[(df_np_observed.index > start_date_feb) & (df_np_observed.index <= end_date_feb)]

In [19]:
f_init = np.zeros(len(df_np_observed.index))

border_flows_obs = {
    'AT-CZ': f_init.copy(), 'AT-DE': f_init.copy(), 'AT-HU': f_init.copy(), 'AT-SI': f_init.copy(), 'BE-FR': f_init.copy(),
    'BE-NL': f_init.copy(), 'CZ-DE': f_init.copy(), 'CZ-PL': f_init.copy(), 'CZ-SK': f_init.copy(), 'DE-FR': f_init.copy(),
    'DE-NL': f_init.copy(), 'HR-HU': f_init.copy(), 'HR-SI': f_init.copy(), 'HU-SK': f_init.copy(), 'HU-RO': f_init.copy(),
    'BE-DE': f_init.copy(), 'DE-PL': f_init.copy(), 'HU-SI': f_init.copy(), 'PL-SK': f_init.copy(), 'AT-SK': f_init.copy()
}

for t, timestamp in enumerate(df_np_observed.index):
    cnes = df_ptdf_obs[df_ptdf_obs.DateTime == timestamp]
    for cne_i, cne in cnes.iterrows():
        eic = df_grid.loc[cne['line_id'], 'eic']
        if df_grid.loc[cne['line_id'], 'tieline'] and eic[:4] == '10T-':
            z1 = eic[4:6]
            z2 = eic[7:9]
            flow = 0
            for zone in zones:
                flow += cne[zone] * df_np_observed.loc[timestamp, zone]
            
            border_flows_obs[z1 + '-' + z2][t] += -1*flow

In [21]:
border_flow_forecasts = []

df_np_forecasts = []
df_np_forecasts.append(pd.read_excel("../cost_curves/np_forecasts/np_forecast_full_nuts_norm_1_w_atc_february.xlsx"))
df_np_forecasts.append(pd.read_excel("../cost_curves/np_forecasts/np_forecast_full_alpha5_norm_1_w_atc_february.xlsx"))
df_np_forecasts.append(pd.read_excel("../cost_curves/np_forecasts/np_forecast_full_naive_norm_1_w_atc_february.xlsx"))

for index, df in enumerate(df_np_forecasts):
    df_np_forecasts[index].index = df_np_observed.index

    border_flows_calc = {
        'AT-CZ': f_init.copy(), 'AT-DE': f_init.copy(), 'AT-HU': f_init.copy(), 'AT-SI': f_init.copy(), 'BE-FR': f_init.copy(),
        'BE-NL': f_init.copy(), 'CZ-DE': f_init.copy(), 'CZ-PL': f_init.copy(), 'CZ-SK': f_init.copy(), 'DE-FR': f_init.copy(),
        'DE-NL': f_init.copy(), 'HR-HU': f_init.copy(), 'HR-SI': f_init.copy(), 'HU-SK': f_init.copy(), 'HU-RO': f_init.copy(),
        'BE-DE': f_init.copy(), 'DE-PL': f_init.copy(), 'HU-SI': f_init.copy(), 'PL-SK': f_init.copy(), 'AT-SK': f_init.copy()
    }

    for t, timestamp in enumerate(df_np_observed.index):
        if 'DateTime' in df_ptdf_forecasts[index].columns:
            cnes = df_ptdf_forecasts[index][df_ptdf_forecasts[index].DateTime == timestamp]
        else:
            cnes = df_ptdf_forecasts[index][df_ptdf_forecasts[index].index == timestamp]
        for cne_i, cne in cnes.iterrows():
            eic = df_grid.loc[cne['line_id'], 'eic']
            if df_grid.loc[cne['line_id'], 'tieline'] and eic[:4] == '10T-':
                z1 = eic[4:6]
                z2 = eic[7:9]
                flow = 0
                for zone in zones:
                    flow += cne[zone] * df_np_forecasts[index].loc[timestamp, zone]

                border_flows_calc[z1 + '-' + z2][t] += -1*flow
        
    border_flow_forecasts.append(border_flows_calc)

In [40]:
flow_error_zones = []
flow_best_cne_selection = []
for border in border_flows_obs.keys():
    
    diff_a10 = []
    diff_a5 = []
    diff_n = []
    
    for t in range(len(border_flows_obs[border])):
        diff_a10_t = border_flows_obs[border][t] - border_flow_forecasts[0][border][t]
        diff_a5_t = border_flows_obs[border][t] - border_flow_forecasts[1][border][t]
        diff_n_t = border_flows_obs[border][t] - border_flow_forecasts[2][border][t]
        if abs(border_flows_obs[border][t]) > 0:
            if abs(border_flow_forecasts[0][border][t]) > 0:
                diff_a10.append(abs(diff_a10_t))
            if abs(border_flow_forecasts[1][border][t]) > 0:
                diff_a5.append(abs(diff_a5_t))
            if abs(border_flow_forecasts[2][border][t]) > 0:
                diff_n.append(abs(diff_n_t))
            
    if len(diff_a10) > 0:
        mean_a10 = np.mean(diff_a10)
    else:
        mean_a10 = 0
        
    if len(diff_a5) > 0:
        mean_a5 = np.mean(diff_a5)
    else:
        mean_a5 = 0
        
    if len(diff_n) > 0:
        mean_n = np.mean(diff_n)
    else:
        mean_n = 0
        
        
    border_flows_denom = np.max(abs(border_flows_obs[border]))

    if border_flows_denom > 0:
        flow_error_zones.append(min(
            (100*mean_a10 / border_flows_denom) if mean_a10 > 0 else 10000,
            (100*mean_a5 / border_flows_denom) if mean_a5 > 0 else 10000,
            #(100*mean_n / border_flows_denom) if mean_n > 0 else 10000,
        ))

        flow_best_cne_selection.append(np.argmin([
            (100*mean_a10 / border_flows_denom) if mean_a10 > 0 else 10000,
            (100*mean_a5 / border_flows_denom) if mean_a5 > 0 else 10000,
            #(100*mean_n / border_flows_denom) if mean_n > 0 else 10000,
        ]))
    else:
        flow_error_zones.append(0)
        flow_best_cne_selection.append(0)

In [53]:
border_distance_comp = np.zeros((len(borders), 3))
border_distance_comp[:, 0] = np.transpose([np.argmin(border_distance[b][0:2]) for b in range(len(borders))])
border_distance_comp[:, 1] = np.transpose([np.argmin(border_distance_c[b][0:2]) for b in range(len(borders))])
border_distance_comp[:, 2] = np.transpose(flow_best_cne_selection)

In [56]:
d_flow_scores = []
d_c_flow_scores = []

for b, border in enumerate(borders):
    print(border, border_distance_comp[b])
    if border_distance_comp[b, 0] == border_distance_comp[b, 2]:
        d_flow_scores.append(border)
    if border_distance_comp[b, 1] == border_distance_comp[b, 2]:
        d_c_flow_scores.append(border)

['AT', 'CZ'] [1. 1. 1.]
['AT', 'DE_LU'] [1. 0. 0.]
['AT', 'HU'] [1. 1. 1.]
['AT', 'SI'] [1. 1. 1.]
['BE', 'FR'] [1. 1. 1.]
['BE', 'NL'] [1. 0. 1.]
['CZ', 'DE_LU'] [1. 0. 1.]
['CZ', 'PL'] [1. 1. 1.]
['CZ', 'SK'] [1. 1. 1.]
['DE_LU', 'FR'] [1. 0. 0.]
['DE_LU', 'NL'] [1. 0. 0.]
['HR', 'HU'] [1. 1. 1.]
['HR', 'SI'] [0. 1. 1.]
['HU', 'SK'] [1. 1. 0.]
['HU', 'RO'] [1. 1. 0.]
['BE', 'DE_LU'] [1. 0. 0.]
['DE_LU', 'PL'] [1. 0. 1.]
['HU', 'SI'] [1. 1. 0.]
['PL', 'SK'] [1. 0. 1.]
['AT', 'SK'] [1. 1. 0.]


In [57]:
print('d score:', len(d_flow_scores))
print('d_c score:', len(d_c_flow_scores))

d score: 11
d_c score: 12


In [60]:
d_flow_scores

[['AT', 'CZ'],
 ['AT', 'HU'],
 ['AT', 'SI'],
 ['BE', 'FR'],
 ['BE', 'NL'],
 ['CZ', 'DE_LU'],
 ['CZ', 'PL'],
 ['CZ', 'SK'],
 ['HR', 'HU'],
 ['DE_LU', 'PL'],
 ['PL', 'SK']]

In [65]:
print('Captured by both: ', 
        set([b[0]+'-'+b[1] for b in d_flow_scores]).intersection([b[0]+'-'+b[1] for b in d_c_flow_scores])
)

print('Captured by d: ', 
        set([b[0]+'-'+b[1] for b in d_flow_scores]).difference([b[0]+'-'+b[1] for b in d_c_flow_scores])
)

print('Captured by d_c: ', 
        set([b[0]+'-'+b[1] for b in d_c_flow_scores]).difference([b[0]+'-'+b[1] for b in d_flow_scores])
)

Captured by both:  {'CZ-SK', 'CZ-PL', 'AT-SI', 'BE-FR', 'AT-CZ', 'AT-HU', 'HR-HU'}
Captured by d:  {'CZ-DE_LU', 'PL-SK', 'BE-NL', 'DE_LU-PL'}
Captured by d_c:  {'HR-SI', 'DE_LU-NL', 'BE-DE_LU', 'AT-DE_LU', 'DE_LU-FR'}


In [66]:
border_distance_c

array([[ 2.5030503 ,  1.95052557,  1.94300534],
       [ 2.51714149,  3.3894666 ,  2.31663778],
       [ 6.21533979,  2.35235134,  1.93489808],
       [ 3.97629701,  3.01642849,  2.1604606 ],
       [ 1.02834721,  0.99320315,  1.225265  ],
       [ 1.07516739,  2.45795476,  2.81654165],
       [ 2.28249437,  3.61236123,  1.81988501],
       [ 1.35751572,  1.31975225,  7.2552231 ],
       [ 3.73056956,  1.8086859 ,  1.87347251],
       [ 1.49806679,  2.93349591,  1.76726159],
       [ 1.10433399,  3.4056856 ,  4.57407529],
       [ 3.27476749,  1.59573875,  1.96166371],
       [ 0.974144  ,  0.94010554,  0.6038694 ],
       [ 6.29267372,  2.75094089,  1.49945325],
       [ 3.45902062,  2.65966019,  4.60345971],
       [ 1.13162067,  2.2582219 ,  1.66936998],
       [ 1.12164156,  2.0481122 , 15.25647766],
       [ 5.80307299,  2.04264162,  1.55551777],
       [ 1.71837878,  1.74319744,  6.16128419],
       [ 8.68248838,  2.24630858,  1.69631699]])

In [ ]:
[1. 1. 1.] x7
[1. 1. 0.] x2 - both wrong

[1. 0. 0.] x4
[1. 0. 1.] x4

[0. 1. 1.]  x 1

[0. 1. 0.] - does not exist